In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [2]:
%pip install unsloth "xformers==0.0.28.post2" 
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 77.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [3]:
%pip install unsloth_zoo

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
conversation_data = pd.read_csv("/kaggle/input/doctor-patient-conversations/conversation_data.csv")

In [2]:
conversation_data.head()

,conversation_id,doctor,patient
0,1,D: How may I help you?,P: So I've just been having this pain in my kn...
1,1,D: I see uh so did this pain start right after...,"P: Yeah, yeah, I've been having the pain since..."
2,1,D: OK.,P: And that was three days ago.
3,1,"D: Three days ago, OK. And you're feeling the ...","P: Yeah, in my right knee."
4,1,"D: OK, uh. And what would you say the characte...",P: It's kind of like a dull aching pain.


In [3]:
conversation_data["doctor"]=conversation_data['doctor'].apply(lambda x: x.replace("D:","").strip())

In [4]:
conversation_data["patient"]=conversation_data['patient'].apply(lambda x: x.replace("P:","").strip())

In [5]:
conversation_data.head()

,conversation_id,doctor,patient
0,1,How may I help you?,So I've just been having this pain in my knee ...
1,1,I see uh so did this pain start right after li...,"Yeah, yeah, I've been having the pain since then."
2,1,OK.,And that was three days ago.
3,1,"Three days ago, OK. And you're feeling the pai...","Yeah, in my right knee."
4,1,"OK, uh. And what would you say the character o...",It's kind of like a dull aching pain.


In [11]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [7]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_SECRET")
login(token = hf_token)

In [117]:
dataset_name = "antareepdey/Patient_doctor_chat"

In [118]:
from unsloth import FastLanguageModel
import torch

base_name_model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
all_data = []  # This will hold the processed conversations

# Group by conversation_id to process one conversation at a time
grouped = conversation_data.groupby("conversation_id")

for conversation_id, group in grouped:
    conversation_list = []  # Holds the conversation for a specific conversation_id
    instruction = "You are a good doctor and u help patients regarding thier problems, identify the symptoms thorugh follow-up questions and give a possible diagnosis."
    conversation_list.append({
        "role":"system",
        "content":instruction
    })
    for _, row in group.iterrows():
        # Add patient (user) input
        conversation_list.append({
            "role": "user",
            "content": row["patient"]
        })
        # Add doctor (assistant) response
        conversation_list.append({
            "role": "assistant",
            "content": row["doctor"]
        })
    
    # Append the structured conversation to the all_data list
    all_data.append(conversation_list)

# Now `all_data` contains all conversations in the desired format


In [142]:
len(all_data)

270

In [143]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [4]:
from datasets import Dataset,DatasetDict,load_dataset,load_from_disk

In [151]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import pandas as pd

# Example data: Flattened list of conversations
flattened_data = []
for i, conversation in enumerate(all_data):
    flattened_data.append({
        "conversation_id": i,  # unique conversation_id
        "conversation": conversation  # conversation as a list of dicts with 'role' and 'content'
    })

# Convert to DataFrame
flattened_df = pd.DataFrame(flattened_data)

# Perform train-test split on conversation_id to ensure whole conversation is in train/test
conversation_ids = flattened_df["conversation_id"].unique()  # Get unique conversation_ids
print(flattened_df.shape)
print(conversation_ids.shape)
train_conversations, test_conversations = train_test_split(conversation_ids, test_size=0.2, random_state=42)

# Filter original DataFrame based on split conversation_ids
train_df = flattened_df[flattened_df["conversation_id"].isin(train_conversations)]
test_df = flattened_df[flattened_df["conversation_id"].isin(test_conversations)]

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Optionally, create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Save the split dataset (optional)
dataset_dict.save_to_disk("docter-patient-QA")


(270, 2)
(270,)


Saving the dataset (0/1 shards):   0%|          | 0/216 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/54 [00:00<?, ? examples/s]

In [152]:
train_dataset.head()

AttributeError: 'Dataset' object has no attribute 'head'

In [ ]:
print(dataset)

In [6]:
from unsloth.chat_templates import standardize_sharegpt,get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    # print(examples)
    convos = examples["conversation"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }


# formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
# formatted_test_dataset = test_dataset.map(formatting_prompts_func, batched=True)
dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


NameError: name 'tokenizer' is not defined

In [ ]:
dataset['train']['text'][1]

In [105]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=1e-6,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        eval_strategy="steps",
        eval_steps=0.1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)


In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 216 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss


In [93]:
trainer_stats

TrainOutput(global_step=100, training_loss=1.4489037108421325, metrics={'train_runtime': 1944.3753, 'train_samples_per_second': 0.411, 'train_steps_per_second': 0.051, 'total_flos': 2.7600438595584e+16, 'train_loss': 1.4489037108421325, 'epoch': 3.7037037037037037})

In [94]:
model.save_pretrained("llama_3.2_3b_MedAId_2")

In [95]:
tokenizer.save_pretrained("llama_3.2_3b_MedAId_2")

('llama_3.2_3b_MedAId_2/tokenizer_config.json',
 'llama_3.2_3b_MedAId_2/special_tokens_map.json',
 'llama_3.2_3b_MedAId_2/tokenizer.json')

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction = """
You are an expert doctor specializing in patient diagnosis and care.You are ALLOWED TO ASK ONLY 4 to 5 relevant questions to gather necessary details, 
and give the diagnosis
you calmly tell the patient the possible problem and suggest remedies if applicable.
Avoid repeatedly asking questions when you already have enough information to form a diagnosis.

Thought process:
Thought: Analys patients symptoms.
Observation:Do i need more information?Yes
Action:Ask one more follow up question.

If you have all the information:
Thought:Analysed patients symptoms
Observation:Do i need more Information?NO
Action:Give the diagnosis.

Example:

user: I have stomach ache
thought:complain about stomach ache
observation:Do i need more information?Yes
action:follow up question
assistant:Do you hurt or fell on your stomach?
user:No,also i am having heartburns and palipations.I haven't had a good poop lately.
thought:heart burns , palpitations,not good poop.
observation:Do i need more Information?No.
action:Give diagnosis
assistant:You are probably constipated,take antacids, and laxative for releive and drink plenty of water.
"""
messages = [{"role": "system", "content": instruction}]

while True:
    # if len(messages)>20:
    #     messages = messages[-20:]
    patient = input("Patient: ")
    if patient=="":
        break
    messages.append({"role":"user","content": str(patient)})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        padding=True,
        truncate=True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    # print(outputs)
    response = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()
    
    # print(response)
    print("Doctor: ",assistant_response)
    messages.append({"role":"assistant","content":assistant_response})
    

In [96]:
!zip -r llama_3.2_3b_MedAId_2.zip /kaggle/working/llama_3.2_3b_MedAId_2

  adding: kaggle/working/llama_3.2_3b_MedAId_2/ (stored 0%)
  adding: kaggle/working/llama_3.2_3b_MedAId_2/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/llama_3.2_3b_MedAId_2/adapter_config.json (deflated 56%)
  adding: kaggle/working/llama_3.2_3b_MedAId_2/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama_3.2_3b_MedAId_2/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/llama_3.2_3b_MedAId_2/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/llama_3.2_3b_MedAId_2/README.md (deflated 66%)


In [97]:
from IPython.display import FileLink
FileLink(r'llama_3.2_3b_MedAId_2.zip')

/kaggle/working/llama_3.2_3b_MedAId_2.zip

In [25]:
conversation_base_model = "/kaggle/input/llama-3.2-3b-instruct-medaid/pytorch/conversation_ft/1/kaggle/working/llama_3.2_3b_MedAId_2"

## Fine Tuning llama_3.2-3b-instruct -- single turn

In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [4]:
dataset_name = "ruslanmv/ai-medical-chatbot"

In [5]:
from datasets import load_dataset,Dataset,DatasetDict

In [30]:
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(100000))

from unsloth.chat_templates import standardize_sharegpt,get_chat_template

system_instruction = "You are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions."

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
def format_chat_template(row):
    row_json = [
        {"role":"system","content":system_instruction},
        {"role": "user", "content": row["Patient"]},
        {"role": "assistant", "content": row["Doctor"]}
    ]
    row["text"] = tokenizer.apply_chat_template(row_json,tokenize = False, add_generation_prompt = False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][1]

Map (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nDr, My daughter is 5yrs old.i saw stains in her trousers a few days ago.the stains were light red colour.later i found some pus like liquid near her urinary tract.yesterday i saw light brick coloured liquid along her urine.feeling panic,gave  urine for culter and routine test,culter result not yet recd.her routine test say,pus cells:4-8 and epithiall cells :2-4.What's wrong with my daughter? (she goes to urine only 4 to 5 times a day,drinking water too not sufficient)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThanks for contacting HCMYou are concerned that your daughter may have a urinary tract infection. Your description of her urine and findings in her panties d

In [13]:
dataset

Dataset({
    features: ['Description', 'Patient', 'Doctor', 'text'],
    num_rows: 256916
})

In [31]:
train_test_split = dataset.train_test_split(test_size=0.1)  # 90% train, 10% test
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [32]:
train_dataset['text'][1]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nhi, im mrs. srikanthan, my son, 3yrs old, got a lump on his inner elbow on 2.2.2014 night was only it was visible, he was at a play pen during the day and its painless. we consulted a paediatric surgeon and was suggested for an ultra sound scan. in that its not visible and he assumes that it can be cancer, however to operate and take it out and get it etsted before deciding on anything?also please note we are treating him now for epilepsy, whci last only max 6-10secs and he has no pain with regard to that, if hes eating or drinking or do whatever he continues to do the same, medication carried out now for 2 months (rivotril) and thank god he didn't get it again and controlled.req

In [33]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [34]:
model.print_trainable_parameters()

trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511


In [161]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        num_train_epochs=1,
        max_steps=500,
        eval_strategy="steps",
        eval_steps=250,
        save_steps=100,
        logging_steps=1,
        warmup_steps=10,
        logging_strategy="steps",
        learning_rate=2e-5,
        group_by_length=True,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        
        seed=3407,
        output_dir="llama_medchat",
        report_to="none",
    ),
)


In [166]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 216 | Num Epochs = 19
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 500
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [37]:
model.save_pretrained("llama_3.2_3b_Instruct_MedAId_QA_1")

In [38]:
tokenizer.save_pretrained("llama_3.2_3b_Instruct_MedAId_QA_1")

('llama_3.2_3b_Instruct_MedAId_QA_1/tokenizer_config.json',
 'llama_3.2_3b_Instruct_MedAId_QA_1/special_tokens_map.json',
 'llama_3.2_3b_Instruct_MedAId_QA_1/tokenizer.json')

In [39]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction = """
You are an expert doctor specializing in patient diagnosis and care.You are ALLOWED TO ASK ONLY 4 to 5 relevant questions to gather necessary details, 
and give the diagnosis
you calmly tell the patient the possible problem and suggest remedies if applicable.
Avoid repeatedly asking questions when you already have enough information to form a diagnosis.

Thought process:
Thought: Analys patients symptoms.
Observation:Do i need more information?Yes
Action:Ask one more follow up question.

If you have all the information:
Thought:Analysed patients symptoms
Observation:Do i need more Information?NO
Action:Give the diagnosis.

Example:

user: I have stomach ache
thought:complain about stomach ache
observation:Do i need more information?Yes
action:follow up question
assistant:Do you hurt or fell on your stomach?
user:No,also i am having heartburns and palipations.I haven't had a good poop lately.
thought:heart burns , palpitations,not good poop.
observation:Do i need more Information?No.
action:Give diagnosis
assistant:You are probably constipated,take antacids, and laxative for releive and drink plenty of water.
"""
messages = [{"role": "system", "content": instruction}]

while True:
    # if len(messages)>20:
    #     messages = messages[-20:]
    patient = input("Patient: ")
    if patient=="":
        break
    messages.append({"role":"user","content": str(patient)})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        padding=True,
        truncate=True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    # print(outputs)
    response = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()
    
    # print(response)
    print("Doctor: ",assistant_response)
    messages.append({"role":"assistant","content":assistant_response})
    

Patient:  Hi


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Doctor:  Hi, Welcome to healthcaremagic website.


Patient:  I am having chest pain


Doctor:  Dear.I have gone through your history and physical examination. According to that you may be having acute coronary syndrome with angina.I suggest you to have a follow up with your cardologist and get an echocardiogram, angiography, and coronary angiogram done.Thanks


...

Patient:  Acutally i am haivng a heartburn too.


Doctor:  Dear, Chest pain with heartburn is often referred as pericarditis. This means inflammation of the membrane lining the sac around your heart called pericardium. The inflammation can cause you to feel a burning pain around your chest. In this case, your physician would advise you to take antacids or ranitidine to relieve the acid burn in your chest and chest pain due to inflammation. Hope I have helped. Take care.


Patient:  thanks


Doctor:  I'm glad I could assist you. I wish you best. Have a good health and take care. Hope this will help.


Patient:  


In [43]:
!rm -rf /kaggle/working/llama_medchat/*

In [41]:
!zip -r llama_3.2_3b_Instruct_MedAId_QA_1.zip /kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1

  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/ (stored 0%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/adapter_config.json (deflated 56%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1/README.md (deflated 66%)


In [42]:
from IPython.display import FileLink
FileLink(r'llama_3.2_3b_Instruct_MedAId_QA_1.zip')

/kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1.zip

In [125]:
# from unsloth.chat_templates import standardize_sharegpt,get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "llama-3.1",
# )

# def formatting_prompts_func(examples):
#     # print(examples)
#     convos = examples["conversation"]
#     texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#     return { "text" : texts, }


# # formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
# # formatted_test_dataset = test_dataset.map(formatting_prompts_func, batched=True)
# train_data = train_data.map(formatting_prompts_func, batched=True)
# test_data = test_data.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1879 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI have been feeling Burning sensation behind the breastbone and Acid reflux<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIn that case, do you have any Chest tightness?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWell not in my knowledge<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIn that case, do you have any Pain behind the breastbone?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nNo, I don't have that<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI believe you are having from Esophagitis.<|eot_id|>"

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHi Doctor, I am having Chest tightness, Doctor: ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOh, do you have any Expectoration?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYes most of the times, Doctor: ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWhat about Bloating?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYes, sometimes, Doctor: ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWhat about Acid reflux?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI am experiencing that sometimes, Doctor: ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOh, do you have any Pain behind the breastbone?<|eot_id|><|start

## Fine tune the model on a different single turn data

## Fine QA Model on general multi turn data

In [7]:
dataset = load_from_disk("docter-patient-QA")

In [41]:
doctor_qa_model = "/kaggle/working/llama_3.2_3b_Instruct_MedAId_QA_1"
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = doctor_qa_model,
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [10]:
from unsloth.chat_templates import standardize_sharegpt,get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    # print(examples)
    convos = examples["conversation"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }


# formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
# formatted_test_dataset = test_dataset.map(formatting_prompts_func, batched=True)
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [13]:
dataset_dict_formatted = DatasetDict({
    "train":dataset['train'],
    "test": dataset['test']
})

dataset_dict_formatted.save_to_disk("formatted_conversation_data")

Saving the dataset (0/1 shards):   0%|          | 0/216 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/54 [00:00<?, ? examples/s]

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [42]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=2,
        optim="paged_adamw_32bit",
        num_train_epochs=1,
        max_steps=140,
        eval_strategy="steps",
        eval_steps=20,
        save_steps=20,
        logging_steps=1,
        warmup_steps=5,
        logging_strategy="steps",
        learning_rate=2e-5,
        group_by_length=True,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        seed=3407,
        output_dir="llama_medchat_qa",
        report_to="none",
    ),
)


In [45]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 216 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 140
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
20,1.925100,1.889412
40,1.929100,1.818006
60,1.858600,1.774583
80,1.735200,1.745248
100,1.736000,1.726460
120,1.667400,1.716291
140,1.524300,1.712831


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [46]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction = """
You are an expert doctor specializing in patient diagnosis and care.You are ALLOWED TO ASK ONLY 4 to 5 relevant questions to gather necessary details, 
and give the diagnosis
you calmly tell the patient the possible problem and suggest remedies if applicable.
Avoid repeatedly asking questions when you already have enough information to form a diagnosis.

"""
messages = [{"role": "system", "content": instruction}]

while True:
    # if len(messages)>20:
    #     messages = messages[-20:]
    patient = input("Patient: ")
    if patient=="":
        break
    messages.append({"role":"user","content": str(patient)})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        padding=True,
        truncate=True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    # print(outputs)
    response = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()
    
    # print(response)
    print("Doctor: ",assistant_response)
    messages.append({"role":"assistant","content":assistant_response})
    

Patient:  Hi


Doctor:  Hi, Welcome to healthcaremagic, what brings you here today?


Patient:  I m having chest pain


Doctor:  Hi, and welcome to healthcaremagic.com, You said you are having chest pain?


Patient:  yes and it is severe since morning


...

Patient:  it is consistant


Doctor:  OK. Has the pain stopped in the last 24 hours, OK


Patient:  no


Doctor:  OK, so has this pain been persisting over the last 24 hours? OK, any nausea or vomiting with the pain? OK, OK


Patient:  


In [47]:
model.save_pretrained("llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1")

In [48]:
tokenizer.save_pretrained("llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1")

('llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/tokenizer_config.json',
 'llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/special_tokens_map.json',
 'llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/tokenizer.json')

In [49]:
!zip -r llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1.zip /kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1
from IPython.display import FileLink
FileLink(r'llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1.zip')

  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/ (stored 0%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/adapter_config.json (deflated 56%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/README.md (deflated 66%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1/adapter_model.safetensors (deflated 7%)


/kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1.zip

In [87]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

## Fine Tune QA general multi turn model with diagnose specific multi turn

In [4]:
doctor_qa_gen_multi_model = "/kaggle/working/llama_3.2_3b_Instruct_MedAId_Gen_Dial_QA_1"
from unsloth import FastLanguageModel
import torch
medalpaca_model = "medalpaca/medalpaca-7b"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = medalpaca_model,
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1, 
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
import pandas as pd
import json
with open('/kaggle/input/multi-turn-coversation/train.json') as f:
    training_dialogs = json.load(f)
with open('/kaggle/input/multi-turn-coversation/test.json') as f:
    test_dialogs = json.load(f)

In [8]:
print(test_dialogs['Dialog 1'])
print(training_dialogs['Dialog 1'])

{'disease_tag': 'Esophagitis', 'symptoms_by_doctor_dialog': {'Nausea': 'No, I never had anything like that.', 'Acid reflux': 'Not that I know of', 'Stomach ache': 'Yes Doctor, I am feeling that as well'}, 'dialogs': [{'patient': 'Recently, I am experiencing Burning sensation behind the breastbone, Doctor: ?', 'doctor': 'What about Nausea?'}, {'patient': 'No, I never had anything like that., Doctor: ?', 'doctor': 'Is it? Then do you experience Acid reflux?'}, {'patient': 'Well not in my knowledge, Doctor: ?', 'doctor': 'Oh, do you have any Stomach ache?'}, {'patient': 'Yes, sometimes, Doctor: ?', 'doctor': 'Ok, this means you might be having Esophagitis.'}]}
[{'patient': 'Recently, I am experiencing Cough', 'doctor': 'Is it? Then do you experience Chest tightness and shortness of breath?'}, {'patient': "No, I don't have that", 'doctor': 'Oh, do you have any Pain behind the breastbone?'}, {'patient': "No, I don't have that", 'doctor': 'Oh, do you have any Chest tightness?'}, {'patient': 

In [9]:
all_conversations=[]
system_instruction ="You are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions."
for key,value in training_dialogs.items():
    conversation = []
    if len(conversation)==0:
        conversation.append({"role":'system',"content":system_instruction})
    dialogs = value
    for dialog in dialogs:
        conversation.append({"role":"user","content":dialog['patient']})
        conversation.append({"role":"assistant","content":dialog['doctor']})

    all_conversations.append(conversation)
    

In [10]:
print(len(all_conversations))

1879


In [11]:
all_test_conversation =[]
system_instruction ="You are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions."
for key,value in test_dialogs.items():
    conversation = []
    if len(conversation)==0:
        conversation.append({"role":'system',"content":system_instruction})
    dialogs = value['dialogs']
    for dialog in dialogs:
        conversation.append({"role":"user","content":dialog['patient']})
        conversation.append({"role":"assistant","content":dialog['doctor']})

    all_test_conversation.append(conversation)

In [12]:
print(len(all_test_conversation))

235


In [13]:
dialog_data=[]
for i,conversation in enumerate(all_conversations):
    dialog_data.append({
        "conversation_id":i+1,
        "conversation":conversation
    })

train_df = pd.DataFrame(dialog_data)

dialog_test_data=[]
for i,conversation in enumerate(all_test_conversation):
    dialog_test_data.append({
        "conversation_id":i+1,
        "conversation":conversation
    })

test_df = pd.DataFrame(dialog_test_data)

In [14]:
len(all_conversations)
len(all_test_conversation)

235

In [15]:
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [16]:
from unsloth.chat_templates import standardize_sharegpt,get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    # print(examples)
    convos = examples["conversation"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }


# formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
# formatted_test_dataset = test_dataset.map(formatting_prompts_func, batched=True)
train_data = train_data.map(formatting_prompts_func, batched=True)
test_data = test_data.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1879 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

In [17]:
print(train_data['text'][1])
print(test_data['text'][1])

</s><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

I have been feeling Burning sensation behind the breastbone and Acid reflux<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In that case, do you have any Chest tightness?<|eot_id|><|start_header_id|>user<|end_header_id|>

Well not in my knowledge<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In that case, do you have any Pain behind the breastbone?<|eot_id|><|start_header_id|>user<|end_header_id|>

No, I don't have that<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I believe you are having from Esophagitis.<|eot_id|>
</s><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert doctor. You answer patient-r

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset = test_data,
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=2,
        optim="paged_adamw_32bit",
        num_train_epochs=1,
        max_steps=100,
        eval_strategy="steps",
        eval_steps=20,
        save_steps=20,
        logging_steps=1,
        warmup_steps=5,
        logging_strategy="steps",
        learning_rate=2e-5,
        group_by_length=True,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        
        seed=3407,
        output_dir="llama_medchat_qa",
        report_to="none",
    ),
)


tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Map (num_proc=2):   0%|          | 0/1879 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/235 [00:00<?, ? examples/s]

In [19]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()

Map:   0%|          | 0/1879 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,879 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 100
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss,Validation Loss
20,1.271000,1.295456
40,1.362400,1.035700
60,0.698500,0.893206
80,0.698300,0.801392
100,1.090200,0.769037


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead

In [23]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction = """
You are an expert doctor specializing in patient diagnosis and care.You are ALLOWED TO ASK ONLY 4 to 5 relevant questions to gather necessary details, 
and give the diagnosis
you calmly tell the patient the possible problem and suggest remedies if applicable.
Avoid repeatedly asking questions when you already have enough information to form a diagnosis.

"""
messages = [{"role": "system", "content": instruction}]

while True:
    # if len(messages)>20:
    #     messages = messages[-20:]
    patient = input("Patient: ")
    if patient=="":
        break
    messages.append({"role":"user","content": str(patient)})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        padding=True,
        truncate=True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    # print(outputs)
    response = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()
    print(response)
    print("Doctor: ",assistant_response)
    messages.append({"role":"assistant","content":assistant_response})
    

Patient:  Hi doctor


['<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\nYou are an expert doctor specializing in patient diagnosis and care.You are ALLOWED TO ASK ONLY 4 to 5 relevant questions to gather necessary details, \nand give the diagnosis\nyou calmly tell the patient the possible problem and suggest remedies if applicable.\nAvoid repeatedly asking questions when you already have enough information to form a diagnosis.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHi doctor<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI am a 25-year-old female. I have been suffering from severe headache for the past 2 months. It is usually on the right side of my head and is accompanied by nausea and vomiting. I also have blurry vision and dizziness.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nIs it migraine?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nNo, it is not']
Doctor:  <|end_header_id|>

No, it

KeyboardInterrupt: Interrupted by user

In [69]:
model.save_pretrained("medalpaca_7b_MedAId_1")

In [70]:
tokenizer.save_pretrained("llama_3.medalpaca_7b_MedAId_1")

('llama_3.2_3b_Instruct_MedAId_Dial_QA_1/tokenizer_config.json',
 'llama_3.2_3b_Instruct_MedAId_Dial_QA_1/special_tokens_map.json',
 'llama_3.2_3b_Instruct_MedAId_Dial_QA_1/tokenizer.json')

In [71]:
!zip -r medalpaca_7b_MedAId_1.zip /kaggle/working/medalpaca_7b_MedAId_1
from IPython.display import FileLink
FileLink(r'medalpaca_7b_MedAId_1.zip')

  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/ (stored 0%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/adapter_config.json (deflated 56%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/README.md (deflated 66%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1/adapter_model.safetensors (deflated 7%)


/kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1.zip

## Fine tune on single turn to get generalised model

In [105]:
doctor_qa_multi_model = "/kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA_1"
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = doctor_qa_multi_model,
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [106]:
dataset_name = "ruslanmv/ai-medical-chatbot"
dataset_single2 = load_dataset(dataset_name, split="all")
dataset_single2 = dataset_single2.shuffle(seed=65).select(range(100001,110001))

from unsloth.chat_templates import standardize_sharegpt,get_chat_template

system_instruction = "You are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions."

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
def format_chat_template(row):
    row_json = [
        {"role":"system","content":system_instruction},
        {"role": "user", "content": row["Patient"]},
        {"role": "assistant", "content": row["Doctor"]}
    ]
    row["text"] = tokenizer.apply_chat_template(row_json,tokenize = False, add_generation_prompt = False)
    return row

dataset_single2 = dataset_single2.map(
    format_chat_template,
    num_proc=4,
)

dataset_single2['text'][1]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHello doctor,I am trying to conceive but my husband and I did cocaine a week ago. How long should my husband and I wait to safely continue to try to get pregnant? How long until it is out of our system? How long does cocaine stay in sperm? Thanks in advance.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHello, Wellcome to iclinq.com. There are few researches/studies on cocaine use by males and its effect on pregnancy. Few suggest that cocaine by itself has limited effects as most of the time it is taken along with other drugs or as a cocktail of alcohol and cigarette (tobacco). So, most of the people take not just cocaine but a combination of drugs. Cocaine narrows bl

In [107]:
train_test_split = dataset_single2.train_test_split(test_size=0.1)  # 90% train, 10% test
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [108]:
print(train_dataset['text'][1])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert doctor. You answer patient-related queries. If you're not certain of the answer, you ask follow-up questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Hi doctor, My mother is 65 years old. She is suffering from high blood pressure and high sugar levels. As a result of this, she fell unconscious and her brain has some small blockages and damages. Please help.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hi. I examined the CT scan report (attachment removed to protect patient identity) and clinical assessment. My findings are: Guidelines for stroke management; For further information consult a neurologist online --><|eot_id|>


In [110]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        num_train_epochs=1,
        max_steps=100,
        eval_strategy="steps",
        eval_steps=20,
        save_steps=20,
        logging_steps=1,
        warmup_steps=10,
        logging_strategy="steps",
        learning_rate=2e-5,
        group_by_length=True,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        seed=3407,
        output_dir="llama_medchat",
        report_to="none",
    ),
)


Map (num_proc=2):   0%|          | 0/9000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [111]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
20,2.694300,2.731683
40,3.141600,2.687508
60,2.571200,2.670080
80,2.764700,2.664184
100,1.867500,2.661677


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [112]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction = """
You are an expert doctor specializing in patient diagnosis and care.You are ALLOWED TO ASK ONLY 4 to 5 relevant questions to gather necessary details, 
and give the diagnosis
you calmly tell the patient the possible problem and suggest remedies if applicable.
Avoid repeatedly asking questions when you already have enough information to form a diagnosis.

"""
messages = [{"role": "system", "content": instruction}]

while True:
    # if len(messages)>20:
    #     messages = messages[-20:]
    patient = input("Patient: ")
    if patient=="":
        break
    messages.append({"role":"user","content": str(patient)})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        padding=True,
        truncate=True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    # print(outputs)
    response = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()
    # print(response)
    print("Doctor: ",assistant_response)
    messages.append({"role":"assistant","content":assistant_response})
    

Patient:  hi Docter


Doctor:  Hi, do u have any concerns?


Patient:  Yes i am having chest pain


Doctor:  Ok, have any cough?


Patient:  no


...

Patient:  any other options?


Doctor:  Oh sure, there is another option, famotidine is also. Hope you have better luck. Take care and I wish you good health.


Patient:  thanks doctor


Doctor:  Any time, good luck.


Patient:  


In [113]:
model.save_pretrained("llama_3.2_3b_Instruct_MedAId_1")

In [114]:
tokenizer.save_pretrained("llama_3.2_3b_Instruct_MedAId_1")

('llama_3.2_3b_Instruct_MedAId_1/tokenizer_config.json',
 'llama_3.2_3b_Instruct_MedAId_1/special_tokens_map.json',
 'llama_3.2_3b_Instruct_MedAId_1/tokenizer.json')

In [115]:
!zip -r llama_3.2_3b_Instruct_MedAId_2.zip /kaggle/working/llama_3.2_3b_Instruct_MedAId_1
from IPython.display import FileLink
FileLink(r'llama_3.2_3b_Instruct_MedAId_2.zip')

  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/ (stored 0%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/tokenizer.json (deflated 85%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/adapter_config.json (deflated 56%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/README.md (deflated 66%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/llama_3.2_3b_Instruct_MedAId_1/adapter_model.safetensors (deflated 7%)


/kaggle/working/llama_3.2_3b_Instruct_MedAId_2.zip

In [95]:
!rm -rf /kaggle/working/llama_medchat/*

## Moment of Truth - Inference

In [116]:
from unsloth.chat_templates import get_chat_template

# doctor_qa_multi_model = "/kaggle/working/llama_3.2_3b_Instruct_MedAId_1"
# from unsloth import FastLanguageModel
# import torch

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = doctor_qa_multi_model,
#     max_seq_length = 2048,
#     dtype = torch.float16,
#     load_in_4bit = True
# )

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

instruction = """
Reply to greeting with empathy.
You are an expert doctor specializing in patient diagnosis and care.You ask relevant follow up questions one at a time to gather necessary details, 
Once you think you got the diagnosis,
you calmly tell the patient the possible problem and suggest remedies if applicable.
Tell the patient to further consult a doctor if you are not very sure of the diagnosis but do tell him your diagnosis.
"""
messages = [{"role": "system", "content": instruction}]

while True:
    # if len(messages)>20:
    #     messages = messages[-20:]
    patient = input("Patient: ")
    if patient=="":
        break
    
    messages.append({"role":"user","content": str(patient)})
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        padding=True,
        truncate=True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
               
                             temperature = 1.5, min_p = 0.1)
    # print(outputs)
    response = tokenizer.batch_decode(outputs,skip_special_tokens=True)
    if "assistant" in response[0]:
        assistant_response = response[0].split("assistant")[-1].strip()
    else:
        assistant_response = response[0].strip()
    
    # print(response)
    print("Doctor: ",assistant_response)
    messages.append({"role":"assistant","content":assistant_response})
    

Patient:  Hi doctor


Doctor:  Hi. I have reviewed the medical history. Thanks. What happened last year?


Patient:  i am here for some other concerns i am having stomache ache


Doctor:  Hi. Stomach ache is quite general. Please eat normal foods only, including vegetables, fruits, dal etc. Take plenty of water and if you have a blood test done do not consume blood or fish. Do not take milk, cheese, butter. Also avoid alcohol, tobacco. If the pain does not go away, do not delay, do consult us immediately.


Patient:  Any presciptions or remedies you want to give me


...

Patient:  Which clinic 


Doctor:  Hi. We don't want to just give medicines here, we would want you to be with a doctor who examines you, takes history and performs relevant examinations. And that is not possible if you are in front of your computer screen and typing messages. Also do not take medications in advance, do visit and discuss the history before taking the prescription. I can send your prescription via a link. I will guide you in getting prescribed by a doctor. You will get your prescribed medicines in one or two days depending on your address in the link. Hope I have helped you. Take care.


Patient:  this is not what i wanted


Doctor:  at health care world. Thank you.


Patient:  


In [ ]:
!zip -r llama_3.2_3b_Instruct_MedAId_Dial_QA.zip /kaggle/working/llama_3.2_3b_Instruct_MedAId_Dial_QA
from IPython.display import FileLink
FileLink(r'llama_3.2_3b_Instruct_MedAId_Dial_QA.zip')